# 作业2 股票数据清洗

同学们好，本次作业的主要内容为使用pandas进行数据预处理。希望这两天你们已经从Python的课程内容中回过神来了。
没有数据的分析是无源之水，能够熟练处理数据是成为合格的数据分析师的基本要求，希望大家在今后多多实战，成为数据大师。

本次作业将使用公开标普500的股票数据。
https://www.kaggle.com/dgawlik/nyse#prices-split-adjusted.csv

作业的形式以问答为主，因为不涉及过长的代码，核心是数据的操作，所以这里没有太多的代码模板可供大家参考。
希望大家能从搜索引擎（google/bing）问答社区（stackoverflow）或者[官方文档](https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html)或者我提供的jupyter notebooks 中找到回答问题需要的方法。
请时刻牢记数据的增删改查四种原子操作，思考我们的问题可以被如何分解。

那么首先，我们要import pandas

In [1]:
import pandas as pd

# 1. 从fundamentals.csv开始！

fundamentals.csv 是这些股票的年报数据

请用数据回答以下问题：

1. S&P500股票在2015年`net income`的均值是多少？最大值比最小值多多少？（每问10分，共计20分）
2. S&P500股票在2016年的固定资产（fixed assets）占总资产(total assets)比例的均值是多少？固定资产占总资产比例最小的股票是的代码（ticker symbol）是什么？（每问10分，共计20分）


In [78]:
fun_df = pd.read_csv('nyse/fundamentals.csv')
fun_df['Period Ending'] = pd.to_datetime(fun_df['Period Ending'])
fun_df = fun_df.set_index('Period Ending')
# 2015年净收入（net income）的均值
mean_net_income = fun_df.loc['2015', 'Net Income'].mean()
print('2015年净收入的均值：{}'.format(mean_net_income))
# 最大值和最小值的差值
difference = fun_df.loc['2015', 'Net Income'].max() - fun_df.loc['2015', 'Net Income'].min()
print('最大值比最小值多：{}'.format(difference)) 

print('\n')

# 2016年的固定资产（fixed assets）占总资产(total assets)比例的均值
proportion_mean = (fun_df.loc['2016', 'Fixed Assets'] / fun_df.loc['2016', 'Total Assets']).mean()
print('固定资产占总资产比例的均值：{}'.format(proportion_mean))
# 比列的最小值
proportion_min = (fun_df.loc['2016', 'Fixed Assets'] / fun_df.loc['2016', 'Total Assets']).min()
symbol_list = []
# 找出比例最小的股票是的代码（ticker symbol）
def find_prop_min(obj):
    for index, row in obj.iterrows():
        proportion = row['Fixed Assets'] / row['Total Assets']
        if proportion == proportion_min:
            symbol_list.append(row['Ticker Symbol'])
    return symbol_list

print('固定资产占总资产比例最小的股票的代码：{}'.format(find_prop_min((fun_df.loc['2016']))))

2015年净收入的均值：1560252997.752809
最大值比最小值多：76922000000.0


固定资产占总资产比例的均值：0.24428208876549706
固定资产占总资产比例最小的股票的代码：['LEN', 'TRV']


# 2. 加入securities.csv~

securities.csv包含了这些股票的基本信息

1. 请列举出各个sector中的加入时间最早的股票名称（10分）
2. 请列举出每一个州中加入时间最晚的股票名称（10分）


In [89]:
sec_df = pd.read_csv('nyse/securities.csv')
sec_df.head()
sec_df['Date first added'] = pd.to_datetime(sec_df['Date first added'])
sec_df = sec_df.set_index('Date first added')
sec_df = sec_df.sort_index()
sec_df.dropna(axis = 0, how = 'any', inplace = True) # 处理缺失数据
# 找出各个sector中的加入时间最早的股票名称
symbol_earliest = []
def find_time_earliest(obj):
    sector_list = []
    for index, row in obj.iterrows():
        if row['GICS Sector'] not in sector_list:
            sector_list.append(row['GICS Sector'])
            symbol_earliest.append(row['Ticker symbol'])
    return symbol_earliest
print('各个sector中的加入时间最早的股票名称：{}'.format(find_time_earliest(sec_df)))

print('\n')

sec_df = sec_df.sort_index(ascending = False)
# 找出各个sector中的加入时间最晚的股票名称
symbol_latest = []
def find_time_latest(obj):
    sector_list = []
    for index, row in obj.iterrows():
        if row['GICS Sector'] not in sector_list:
            sector_list.append(row['GICS Sector'])
            symbol_latest.append(row['Ticker symbol'])
    return symbol_latest
print('各个sector中的加入时间最晚的股票名称：{}'.format(find_time_latest(sec_df)))

各个sector中的加入时间最早的股票名称：['ABT', 'HON', 'SHW', 'SLB', 'GIS', 'MCD', 'HPQ', 'JPM', 'DUK', 'T', 'AVB']


各个sector中的加入时间最晚的股票名称：['IDXX', 'MAA', 'COTY', 'CHTR', 'LNT', 'FTV', 'ALB', 'AJG', 'GPN', 'CXO', 'LVLT']


# 3. merge!

现在你需要同时处理来自两个表中的信息了

1. 请思考，合并两个表的信息的时候，我们应该用什么样的准则对其它们（10分）
2. 请列举每个sector在2013-2016年累计Research&Development的总投入（10分）
3. 请列举出每个sector中，在2013-2016年累计Research&development投入最大的3家公司的名称以及投入的数值（20分）

1、若表格之间有共同的列，则对共同的列进行合并
2、若表格之间没有共同的列，则尽量取表格的并集

In [46]:
fun_df = pd.read_csv('nyse/fundamentals.csv')
sec_df = pd.read_csv('nyse/securities.csv')
fun_df.columns = fun_df.columns.str.lower() # 将所有列名小写化
sec_df.columns = sec_df.columns.str.lower() # 将所有列名小写化
new_df = pd.merge(fun_df, sec_df, on = 'ticker symbol', how = 'inner') # 按股票代码（ticker symbol）进行合并
new_df['period ending'] = pd.to_datetime(new_df['period ending'])
new_df = new_df.set_index('period ending')

new_df = new_df.loc[(new_df.index > '2013') & (new_df.index < '2016')] # 筛选出2013-2016年的数据

total_invest = dict()
# 对sector进行迭代，找出每个sector在2013-2016年累计Research&Development的总投入
for sector in new_df['gics sector'].unique():
    total_invest[sector] = new_df[new_df['gics sector'] == sector]['research and development'].sum()
    print('sector {}在2013-2016年累计Research&Development的总投入：{}'.format(sector, total_invest[sector]))
    invest_largest = dict()
    # 对industry进行迭代，找出投入最大的3家公司的名称及投入数值
    for industry in new_df[new_df['gics sector'] == sector]['gics sub industry']:
        invest_largest[industry] = new_df[new_df['gics sub industry'] == industry]['research and development'].sum()
    invest_largest = sorted(invest_largest.items(), key = lambda d: d[1], reverse = True)
    print('该sector中投入最大的3家公司的名称及投入数值：\n{}\n'.format(invest_largest[0:3]))

sector Industrials在2013-2016年累计Research&Development的总投入：42658899000.0
该sector中投入最大的3家公司的名称及投入数值：
[('Industrial Conglomerates', 18063500000.0), ('Construction & Farm Machinery & Heavy Trucks', 11606200000.0), ('Aerospace & Defense', 10097000000.0)]

sector Consumer Discretionary在2013-2016年累计Research&Development的总投入：1860178000.0
该sector中投入最大的3家公司的名称及投入数值：
[('Consumer Electronics', 1187087000.0), ('Leisure Products', 673091000.0), ('Automotive Retail', 0.0)]

sector Information Technology在2013-2016年累计Research&Development的总投入：234483864000.0
该sector中投入最大的3家公司的名称及投入数值：
[('Semiconductors', 69615189000.0), ('Systems Software', 34786269000.0), ('Internet Software & Services', 25462846000.0)]

sector Health Care在2013-2016年累计Research&Development的总投入：150625581000.0
该sector中投入最大的3家公司的名称及投入数值：
[('Pharmaceuticals', 72059000000.0), ('Biotechnology', 42167895000.0), ('Health Care Equipment', 18655096000.0)]

sector Consumer Staples在2013-2016年累计Research&Development的总投入：4833577000.0
该sector中投入最大的3家公司的名称及

# 4. 现在让我们来看看更加复杂的数据

请导入price.csv，然后结合你的聪明才智回答以下问题（附加题，40分）

假设你是某基金公司的老板，现在对于每只股票，你都专门安排了一位负责它的交易员。公司规定每一位交易员手中的资金要么全部买入要么全部卖出（空仓，转化为现金）。假设2016年每一位交易员手中都有10000美元，假设他们都能够看到2016年全年的数据，假设他们都能抓住每一次机会，那么请问2016年底时，赚钱最多的股票是哪一只，赚了多少钱？